In [158]:
import requests
import pandas as pd
import json

# Replace these with your repository details
owner = 'lucamenotti'
repo = '5v5s'
branch = 'main'
token = ''


url = f"https://api.github.com/repos/{owner}/{repo}/contents"

# Make a request to the GitHub API
headers = {'Authorization': f'token {token}'}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    contents = response.json()
    # Filter out the JSON files and get their raw URLs
    json_urls = [file['download_url'] for file in contents if file['name'].endswith('.json')]
else:
    print(f"Failed to fetch files: {response.status_code}")
    json_urls = []

# Read the JSON files into DataFrames
dataframes = [pd.read_json(url) for url in json_urls]

# Combine the DataFrames into a single DataFrame (if necessary)
combined_df = pd.concat(dataframes, ignore_index=True)


In [159]:
hold = []
for i in combined_df['participants']:
    hold.append(pd.Series(i))

df = combined_df.merge(pd.concat(hold, axis = 1).T, left_index= True, right_index=True,how = 'inner').drop(columns= ['participants', 'TOTAL_PLAYER_SCORE','TOTAL_SCORE_RANK','OBJECTIVE_PLAYER_SCORE','PUUID']).fillna(0)

In [ ]:
player_aliases = {
    'EnderDragon7474': 'Ashalo1',
    'Ashalo1': 'Ashalo1',
    'RaVe B00ST': 'Bryant',
    'CoomerHashira': 'Bryant',
    'Item Diversity': 'BloodStainedRose',
    'BigPeTe1239': 'DookieDemon12'
    # Add more mappings as needed
}

In [160]:
df

,matchId,gameDuration,gameVersion,ALL_IN_PINGS,ASSIST_ME_PINGS,ASSISTS,BAIT_PINGS,BARON_KILLS,BARRACKS_KILLED,BARRACKS_TAKEDOWNS,...,WARD_KILLED,WARD_PLACED,WARD_PLACED_DETECTOR,WAS_AFK,WAS_AFK_AFTER_FAILED_SURRENDER,WAS_EARLY_SURRENDER_ACCOMPLICE,WAS_LEAVER,WAS_SURRENDER_DUE_TO_AFK,WIN,COMBAT_PLAYER_SCORE
0,4146140154,1468845,11.24.414.4003,0,0,4,0,0,1,1,...,3,2,2,0,0,0,0,0,Win,0
1,4146140154,1468845,11.24.414.4003,0,0,4,0,0,0,0,...,2,11,0,0,0,0,0,0,Win,0
2,4146140154,1468845,11.24.414.4003,0,0,13,0,0,0,1,...,4,17,2,0,0,0,0,0,Win,0
3,4146140154,1468845,11.24.414.4003,0,0,6,0,0,0,0,...,2,9,3,0,0,0,0,0,Win,0
4,4146140154,1468845,11.24.414.4003,0,0,7,0,0,0,1,...,1,4,2,0,0,0,0,0,Win,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,5030946055,1736341,14.13.596.7996,0,1,21,0,0,1,1,...,2,2,0,0,0,0,0,0,Win,0
576,5030946055,1736341,14.13.596.7996,2,0,5,0,0,1,1,...,2,12,2,0,0,0,0,0,Win,0
577,5030946055,1736341,14.13.596.7996,0,0,29,0,0,0,1,...,4,29,3,0,0,0,0,0,Win,0
578,5030946055,1736341,14.13.596.7996,0,3,7,0,0,0,0,...,1,14,3,0,0,0,0,0,Win,0


In [161]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

['ALL_IN_PINGS',
 'ASSIST_ME_PINGS',
 'BAIT_PINGS',
 'BARRACKS_TAKEDOWNS',
 'BASIC_PINGS',
 'COMMAND_PINGS',
 'DANGER_PINGS',
 'ENEMY_MISSING_PINGS',
 'ENEMY_VISION_PINGS',
 'GET_BACK_PINGS',
 'HOLD_PINGS',
 'HORDE_KILLS',
 'HQ_TAKEDOWNS',
 'ID',
 'LARGEST_ABILITY_DAMAGE',
 'LARGEST_ATTACK_DAMAGE',
 'LAST_TAKEDOWN_TIME',
 'Missions_ChampionsKilled',
 'Missions_CreepScore',
 'Missions_GoldFromStructuresDestroyed',
 'Missions_GoldFromTurretPlatesTaken',
 'Missions_HealingFromLevelObjects',
 'Missions_MinionsKilled',
 'Missions_TurretPlatesDestroyed',
 'NEED_VISION_PINGS',
 'ON_MY_WAY_PINGS',
 'PLAYER_AUGMENT_1',
 'PLAYER_AUGMENT_2',
 'PLAYER_AUGMENT_3',
 'PLAYER_AUGMENT_4',
 'PLAYER_SUBTEAM',
 'PLAYER_SUBTEAM_PLACEMENT',
 'PUSH_PINGS',
 'RETREAT_PINGS',
 'RIFT_HERALD_KILLS',
 'TOTAL_TIME_CROWD_CONTROL_DEALT_TO_CHAMPIONS',
 'TURRET_TAKEDOWNS',
 'VISION_CLEARED_PINGS',
 'WAS_LEAVER',
 'WAS_SURRENDER_DUE_TO_AFK']

In [162]:
for i in df.columns:
    df[i] = pd.to_numeric(df[i], errors = 'ignore')


In [163]:
multi_type = []
for col in df.columns:
    if df[col].apply(type).nunique() != 1:
        multi_type.append(col)
multi_type

[]

In [168]:
df.groupby('NAME').sum()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
adostraa           400
TylerToTheT212     397
IbelieveMeYou      341
nigeriacoc         291
Ashalo1            290
Dyneliex           136
DonutDude17        126
GasterBat          124
ihyash             112
Babar               74
Little Bro Eman     63
RaVe B00ST          60
dizzycat11          58
MilkLL              56
Jickens             55
DookieDemon12       49
uWumnati            45
CoomerHashira       32
batlife10           31
p1eissoawesome      29
Item Diversity      28
somethin idk        15
MonkeyMan4912       11
trunksthefuture     11
Su5hiS               8
Rin Reforged         8
TinyTiger            7
takoyakiii           7
aeti                 6
kebinlee             5
Mossidy              5
EnderDragon7474      5
BigPeTe1239          2
Detraplex            1
Name: CHAMPIONS_KILLED, dtype: int64

In [183]:
temp = df.groupby('matchId').sum()[['NAME']]
temp[temp['NAME'].str.contains('adostraa') != True].index

df[(df['matchId'] == 4939492680) | (df['matchId'] == 4939565193)]['NAME']

480    trunksthefuture
481      IbelieveMeYou
482         RaVe B00ST
483          batlife10
484      DookieDemon12
485            Ashalo1
486             ihyash
487        DonutDude17
488          Detraplex
489         nigeriacoc
490    trunksthefuture
491           uWumnati
492     TylerToTheT212
493             ihyash
494         RaVe B00ST
495            Ashalo1
496             Su5hiS
497        DonutDude17
498      IbelieveMeYou
499         nigeriacoc
Name: NAME, dtype: object

In [184]:
df.groupby('NAME').count()['CHAMPIONS_KILLED'].sort_values(ascending = False)

NAME
nigeriacoc         57
adostraa           56
TylerToTheT212     52
IbelieveMeYou      50
Ashalo1            45
ihyash             38
DonutDude17        34
GasterBat          31
batlife10          25
uWumnati           24
RaVe B00ST         22
Dyneliex           21
Babar              19
DookieDemon12      19
MilkLL             12
Jickens             9
Little Bro Eman     8
somethin idk        8
dizzycat11          7
Item Diversity      6
CoomerHashira       5
p1eissoawesome      5
MonkeyMan4912       4
Su5hiS              3
Mossidy             3
takoyakiii          3
aeti                3
Rin Reforged        2
TinyTiger           2
kebinlee            2
trunksthefuture     2
Detraplex           1
BigPeTe1239         1
EnderDragon7474     1
Name: CHAMPIONS_KILLED, dtype: int64

In [196]:
def standardize (series):
    return (series - series.mean())/series.std()

temp_df = df
standardize(temp_df[['NAME','TOTAL_DAMAGE_DEALT']].groupby('NAME').mean()).sort_values('TOTAL_DAMAGE_DEALT', ascending = False)


,TOTAL_DAMAGE_DEALT
NAME,
trunksthefuture,2.713592
CoomerHashira,1.821351
adostraa,1.279842
Little Bro Eman,1.167286
dizzycat11,1.113334
Jickens,0.823037
Dyneliex,0.711967
Ashalo1,0.694629
IbelieveMeYou,0.633488


In [165]:
for col in df.columns:
    print(col)

matchId
gameDuration
gameVersion
ALL_IN_PINGS
ASSIST_ME_PINGS
ASSISTS
BAIT_PINGS
BARON_KILLS
BARRACKS_KILLED
BARRACKS_TAKEDOWNS
BASIC_PINGS
BOUNTY_LEVEL
CHAMPION_MISSION_STAT_0
CHAMPION_MISSION_STAT_1
CHAMPION_MISSION_STAT_2
CHAMPION_MISSION_STAT_3
CHAMPIONS_KILLED
CHAMPION_TRANSFORM
COMMAND_PINGS
CONSUMABLES_PURCHASED
DANGER_PINGS
DOUBLE_KILLS
DRAGON_KILLS
ENEMY_MISSING_PINGS
ENEMY_VISION_PINGS
EXP
FRIENDLY_DAMPEN_LOST
FRIENDLY_HQ_LOST
FRIENDLY_TURRET_LOST
GAME_ENDED_IN_EARLY_SURRENDER
GAME_ENDED_IN_SURRENDER
GET_BACK_PINGS
GOLD_EARNED
GOLD_SPENT
HOLD_PINGS
HORDE_KILLS
HQ_KILLED
HQ_TAKEDOWNS
ID
INDIVIDUAL_POSITION
ITEM0
ITEM1
ITEM2
ITEM3
ITEM4
ITEM5
ITEM6
ITEMS_PURCHASED
KEYSTONE_ID
KILLING_SPREES
LARGEST_ABILITY_DAMAGE
LARGEST_ATTACK_DAMAGE
LARGEST_CRITICAL_STRIKE
LARGEST_KILLING_SPREE
LARGEST_MULTI_KILL
LAST_TAKEDOWN_TIME
LEVEL
LONGEST_TIME_SPENT_LIVING
MAGIC_DAMAGE_DEALT_PLAYER
MAGIC_DAMAGE_DEALT_TO_CHAMPIONS
MAGIC_DAMAGE_TAKEN
MINIONS_KILLED
Missions_ChampionsKilled
Missions_Creep